In [1]:
%pip install tiktoken pymupdf -q

Note: you may need to restart the kernel to use updated packages.


# 필수 패키지 설치

#### src.processors.document_processor.py
HWP를 PDF로 변환하여 처리하는 코드 추가
- LibreOffice + unoconv(pyuno) 변환을 위한 환경 설정 필요

In [ ]:
# -*- coding: utf-8 -*-
import hashlib
from pathlib import Path
from typing import Optional, Dict, Any, List, Tuple
import tiktoken

try:
    import pymupdf
    import pymupdf4llm
    PYMUPDF_AVAILABLE = True
except ImportError:
    PYMUPDF_AVAILABLE = False

from src.config import get_config
from src.utils.logging_config import get_logger
from tqdm.notebook import tqdm

import importlib
from src.db import documents_db
importlib.reload(documents_db)
from src.db.documents_db import DocumentsDB

class DocumentProcessor:
    """
    문서 처리 클래스

    주요 기능:
    - PDF 파일을 Markdown 형식으로 변환
    - 변환된 데이터를 데이터베이스에 저장
    - 파일 해시 계산

    사용 예:
        processor = DocumentProcessor()
        file_hash = processor.process_pdf("example.pdf")
    """

    def __init__(self, db_path: Optional[str] = None, config=None, progress_callback=None):
        """
        DocumentProcessor 초기화

        Args:
            db_path (Optional[str]): 데이터베이스 파일 경로 (기본값: config에서 로드)
            config: 설정 객체 (기본값: get_config() 호출)
            progress_callback: 진행 상황 콜백 함수
        """
        # 설정 객체 로드 (기본값: get_config 함수 호출)
        self.config = config or get_config()
        # 로깅 객체 초기화
        self.logger = get_logger('[DOCP]')
        
        # 데이터베이스 경로 설정 (기본값: config에서 로드)
        if db_path is None:
            db_path = self.config.DOCUMENTS_DB_PATH

        # 데이터베이스 객체 초기화
        self.docs_db = DocumentsDB(db_path)
        # 토크나이저 초기화 (OpenAI 모델 기반)
        self.tokenizer = tiktoken.encoding_for_model(self.config.OPENAI_TOKENIZER_MODEL)
        # 진행 상황 콜백 함수 설정
        self.progress_callback = progress_callback

        # 페이지 마커 덤프 디렉토리 생성
        if self.config.MARKER_DUMP_ENABLED:
            self.marker_dump_path = Path(self.config.MARKER_DUMP_PATH)
            self.marker_dump_path.mkdir(parents=True, exist_ok=True)
            self.logger.info(f"마커 덤프 활성화: {self.marker_dump_path}")
            self._create_missing_dump_files()
        else:
            self.marker_dump_path = None
    
        # 초기화 완료 메시지 로깅
        self.logger.info(f"DocumentProcessor 초기화 완료 (DB: {db_path})")

    def _create_missing_dump_files(self) -> None:
        """
        DB에 있는 문서 중 덤프 파일이 없는 문서의 마커 덤프 파일 생성
        """
        if not self.config.MARKER_DUMP_ENABLED:
            self.logger.debug("마커 덤프 비활성화됨")
            return
        
        all_docs = self.docs_db.get_documents_all()
        
        if not all_docs:
            self.logger.debug("DB에 문서 없음")
            return
        
        created_count = 0
        for doc in all_docs:
            file_name = doc.get('file_name')
            text_content = doc.get('text_content')
            
            if not file_name or not text_content:
                continue
            
            if self._save_marker_dump_file(file_name, text_content):
                created_count += 1
        
        if created_count > 0:
            self.logger.info(f"기존 문서 덤프 파일 생성: {created_count}개")
            
    def _save_marker_dump_file(self, file_name: str, text_content: str) -> bool:
        """
        단일 문서의 마커 덤프 파일 저장
        
        Args:
            file_name: 문서 파일명
            text_content: 마크다운 텍스트 내용
        
        Returns:
            bool: 파일 생성 여부 (이미 존재하면 False)
        """
        if not self.config.MARKER_DUMP_ENABLED or not self.marker_dump_path:
            return False
        
        # 확장자 처리: .md면 그대로, 아니면 .md로 변경
        file_path = Path(file_name)
        if file_path.suffix.lower() == '.md':
            dump_file_name = file_path.name
        else:
            dump_file_name = file_path.stem + '.md'
        
        dump_file_path = self.marker_dump_path / dump_file_name
        
        if dump_file_path.exists():
            return False
        
        dump_file_path.write_text(text_content, encoding='utf-8')
        self.logger.debug(f"마커 덤프 저장: {dump_file_path}")
        return True

    def clean_markdown_text(self, text: str) -> str:
        """
        Markdown 텍스트 전처리

        Args:
            text: 원본 텍스트

        Returns:
            str: 전처리된 텍스트
        """
        import re

        # 공백 및 탭을 단일 공백으로 변환
        text = re.sub(r'[ \t]+', ' ', text)
        # 연속된 세 줄 이상의 개행을 두 줄로 축소
        text = re.sub(r'\n{3,}', '\n\n', text)
        # 각 줄의 앞뒤 공백 제거
        lines = [line.strip() for line in text.split('\n')]
        # 줄 단위로 다시 합침
        text = '\n'.join(lines)

        return text.strip()

    def calculate_file_hash(self, file_path: str) -> str:
        """
        파일의 SHA-256 해시를 계산합니다.

        Args:
            file_path (str): 해시를 계산할 파일 경로

        Returns:
            str: 파일의 SHA-256 해시값
        """
        # 파일을 바이너리 모드로 열고, 내용을 읽어 SHA-256 해시 계산
        with open(file_path, 'rb') as f:
            return hashlib.sha256(f.read()).hexdigest()

    def markdown_with_progress(self, pdf_path: str) -> Tuple[List[Dict], int]:
        """
        PDF를 페이지별로 Markdown 변환 (진행 상황 표시)
        
        Args:
            pdf_path (str): PDF 파일 경로
        
        Returns:
            List[Dict]: [{'page_num': int, 'content': str}, ...]
        """
        # 파일 이름 추출
        file_name = Path(pdf_path).name
        total_pages = 0
        
        # PDF 파일 열기
        with pymupdf.open(pdf_path) as doc:
            total_pages = len(doc)  # 총 페이지 수 계산

        pages_data = []
        # 진행 상황 표시를 위한 tqdm 초기화
        with tqdm(total=total_pages, desc="PDF to Markdown", unit="page") as pbar:
            for page_num in range(total_pages):
                try:
                    # 페이지를 Markdown 형식으로 변환
                    markdown = pymupdf4llm.to_markdown(
                        doc=pdf_path,
                        pages=[page_num]
                    )
                    
                    # 변환된 Markdown 텍스트 전처리
                    markdown = self.clean_markdown_text(markdown)
                    
                    # 빈 페이지 처리
                    if not markdown.strip():
                        status = 'empty'
                        markdown = self.config.EMPTY_PAGE_MARKER
                        pbar_msg = f"빈 페이지: {page_num + 1}"
                    else:
                        status = 'processing'
                        pbar_msg = f"페이지 {page_num + 1} len={len(markdown)}"
                    
                    # 페이지 데이터 저장
                    pages_data.append({
                        'page_num': page_num + 1,
                        'content': markdown
                    })
                    
                    # 진행 상황 콜백 호출
                    if self.progress_callback:
                        self.progress_callback({
                            'file_name': file_name,
                            'current_page': page_num + 1,
                            'total_pages': total_pages,
                            'page_content_length': len(markdown),
                            'status': status,
                            'error': ""
                        })

                except Exception as e:
                    # 예외 발생 시 경고 메시지 로깅 및 실패 처리
                    pbar_msg = f"페이지 {page_num + 1} 실패: {e}"
                    self.logger.warning(pbar_msg)
                    
                    pages_data.append({
                        'page_num': page_num + 1,
                        'content': self.config.ERROR_PAGE_MARKER
                    })
                    
                    if self.progress_callback:
                        self.progress_callback({
                            'file_name': file_name,
                            'current_page': page_num + 1,
                            'total_pages': total_pages,
                            'page_content_length': 0,
                            'status': 'failed',
                            'error': str(e)
                        })
                finally:
                    # 진행 상황 업데이트
                    pbar.set_postfix_str(pbar_msg)
                    pbar.update(1)
        
        return pages_data, total_pages

    def process_pdf(self, pdf_path: str, pdf_name: Optional[str] = None) -> Optional[str]:
        """
        PDF 파일을 처리하여 Markdown으로 변환하고 DB에 저장합니다.

        Args:
            pdf_path (str): 처리할 PDF 파일 경로
            pdf_name (Optional[str]): 처리할 PDF 파일 이름 (기본값: None)
                데이터베이스에 저장할 때 사용됩니다, 고유 해야 합니다. 만약 동일한 이름이라면 경로를 추가하세요.

        Returns:
            Optional[str]: 처리된 파일의 해시값 (실패 시 None 반환)
        """
        # PyMuPDF 설치 여부 확인
        if not PYMUPDF_AVAILABLE:
            self.logger.error("PyMuPDF/pymupdf4llm 미설치: pip install pymupdf pymupdf4llm")
            return None

        # PDF 파일 경로 확인
        pdf_file = Path(pdf_path)
        if not pdf_file.exists() or not pdf_file.is_file():
            self.logger.error(f"파일 없음: {pdf_path}")
            return None
        
        # 파일 이름 설정
        if pdf_name is None:
            pdf_name = pdf_file.name

        self.logger.info(f"PDF 처리 시작: {pdf_file.name}")

        # 파일 크기 및 해시 계산
        file_size = pdf_file.stat().st_size
        file_hash = self.calculate_file_hash(pdf_path)
        self.logger.debug(f"파일 해시: {file_hash[:16]}...")

        # 중복 검사: file_name과 file_hash가 모두 일치하는 문서 검색
        existing_docs = self.docs_db.search_documents(file_hash)
        if existing_docs:
            self.logger.info(f"이미 처리된 파일 (skip): {pdf_name}, hash={file_hash[:16]}...")
            return file_hash

        # PDF를 Markdown으로 변환
        pages_data, total_pages = self.markdown_with_progress(pdf_path)
        
        # 변환된 페이지 데이터를 하나의 텍스트로 합침
        all_content = []
        for page_data in pages_data:
            page_num = page_data['page_num']
            content = page_data['content']
            
            is_empty = (content == self.config.EMPTY_PAGE_MARKER)
            
            if not is_empty:
                page_content = f"{self.config.PAGE_MARKER_FORMAT.format(page_num=page_num)}\n\n{content}"
            else:
                page_content = content
            
            all_content.append(page_content)
            
        text_content = '\n'.join(all_content)
        text_content = self.clean_markdown_text(text_content)
        
        # 데이터베이스에 저장
        self.docs_db.insert_text_content(
            file_name=pdf_name,
            file_hash=file_hash,
            total_pages=total_pages,
            file_size=file_size,
            text_content=text_content
        )
        
        # 페이지 마커 덤프 저장
        if self._save_marker_dump_file(pdf_name, text_content):
            self.logger.info(f"마커 덤프 생성: {Path(pdf_name).stem}.md")
            
            if not self.marker_dump_path.exists():
                self.marker_dump_path.write_text(text_content, encoding='utf-8')
                self.logger.info(f"마커 덤프 저장: {self.marker_dump_path}")

        self.logger.info(
            f"PDF 처리 완료: {pdf_file.name} "
            f"({len(pages_data)} 페이지, {file_size / 1024:.1f}KB), 변환문자수={len(text_content):,}"
        )
        return file_hash

#### src.db.documents_db

In [ ]:
# -*- coding: utf-8 -*-
import sqlite3
from datetime import datetime
from pathlib import Path
from typing import Optional, List, Dict, Any
from src.utils.logging_config import get_logger
logger = get_logger('[DOCDB]')

class DocumentsDB:
    """
    마크다운 문서 데이터베이스 관리 클래스

    주요 기능:
    - 마크다운 파일 정보를 SQLite 데이터베이스에 저장
    - 데이터베이스 테이블 생성 및 관리
    - 문서 통계 정보 제공

    사용 예:
        db = DocumentsDB("data/documents.db")
        db.insert_text_content(file_hash, file_name, total_pages, file_size, text_content)
        stats = db.get_document_stats()
    """

    def __init__(self, db_path: str = 'data/documents.db'):
        """
        DocumentsDB 초기화

        Args:
            db_path (str): 데이터베이스 파일 경로 (기본값: 'data/documents.db')
        """
        self.USER_VERSION = 1
        self.logger = get_logger('[DOCDB]')
        self.db_path = Path(db_path)
        self.db_path.parent.mkdir(parents=True, exist_ok=True)  # 데이터베이스 경로 생성
        self._create_tables()  # 테이블 생성

    def _get_connection(self):
        """
        SQLite 데이터베이스 연결 생성

        Returns:
            sqlite3.Connection: 데이터베이스 연결 객체
        """
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row  # 결과를 딕셔너리 형태로 반환
        return conn

    def _create_tables(self):
        """
        데이터베이스 테이블 및 인덱스 생성

        - TB_DOCUMENTS: 마크다운 파일 정보를 저장하는 테이블
            - file_hash: 파일의 고유 해시값 (기본 키, 중복 불가)
            - file_name: 파일 이름 (경로 포함, 고유 해야함)
            - total_pages: 원본 파일의 총 페이지 수
            - file_size: 원본 파일 크기 (바이트 단위)
            - text_content: 변환된 파일의 텍스트 콘텐츠 (텍스트, 마크다운, HTML 등)
            - created_at: 레코드 생성 시각 (기본값: 현재 시각)
            - updated_at: 레코드 수정 시각 (기본값: 현재 시각)
        
        - 인덱스:
            - idx_file_name: file_name 컬럼에 대한 인덱스
            - idx_created_at: created_at 컬럼에 대한 시간 기반 검색 최적화를 위한 인덱스
        """
        with self._get_connection() as conn:
            cursor = conn.cursor()
            # TB_DOCUMENTS 테이블 생성
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS TB_DOCUMENTS (
                    file_hash TEXT PRIMARY KEY,  -- 파일 고유 해시값 (중복 불가)
                    file_name TEXT NOT NULL,    -- 파일 이름 (변환 전 이름, 고유 해야함)
                    total_pages INTEGER NOT NULL,  -- 원본 파일 총 페이지 수
                    file_size INTEGER NOT NULL,    -- 원본 파일 크기 (바이트 단위)
                    text_content TEXT,             -- 변환된 파일의 텍스트 콘텐츠
                    created_at TIMESTAMP DEFAULT (datetime('now', '+9 hours')),  -- 생성 시각
                    updated_at TIMESTAMP DEFAULT (datetime('now', '+9 hours'))   -- 수정 시각
                )
            """)
            # 트리거를 사용해 updated_at 컬럼 자동 업데이트
            cursor.execute("""
                CREATE TRIGGER IF NOT EXISTS update_updated_at
                AFTER UPDATE ON TB_DOCUMENTS
                FOR EACH ROW
                BEGIN
                    UPDATE TB_DOCUMENTS
                    SET updated_at = CURRENT_TIMESTAMP
                    WHERE file_hash = OLD.file_hash;
                END;
            """)            
            # 추후 프로그램 고도화시 마이 그레이션 에서 검토 한다.
            # # 인덱스 생성 (file_name)
            # cursor.execute("""
            #     CREATE INDEX IF NOT EXISTS idx_file_name
            #     ON TB_DOCUMENTS (file_name)
            # """)
            # # created_at 인덱스 생성
            # cursor.execute("""
            #     CREATE INDEX IF NOT EXISTS idx_created_at
            #     ON TB_DOCUMENTS (created_at)
            # """)
            
            # PRAGMA user_version을 사용한 DB 버전 관리
            cursor.execute("PRAGMA user_version")
            db_version = cursor.fetchone()[0]
            
            if db_version == 0:
                # 초기화: user_version이 0이면 현재 버전으로 설정
                cursor.execute(f"PRAGMA user_version = {self.USER_VERSION}")
                self.logger.info(f"데이터베이스 버전 초기화: {self.USER_VERSION}")
            elif db_version != self.USER_VERSION:
                self.logger.warning(f"데이터베이스 버전 불일치: {db_version} → {self.USER_VERSION}")
                # 마이그레이션 로직 추가 가능
                self._migrate_database(db_version, self.USER_VERSION)

            conn.commit()
            
    def _migrate_database(self, old_version: int, new_version: int):
        """
        데이터베이스 마이그레이션 로직
        추후 버전 업그레이드 시 필요한 마이그레이션 로직을 여기에 구현
        
        Args:
            old_version (int): 기존 데이터베이스 버전
            new_version (int): 새로운 데이터베이스 버전
        """
        #with self._get_connection() as conn:
            #cursor = conn.cursor()
            #self.logger.info(f"데이터베이스 마이그레이션 시작: {old_version} → {new_version}")
            
            # if old_version == 1 and 2 == new_version:
                
            #     # 버전 2로 마이그레이션
            #     cursor.execute("ALTER TABLE TB_DOCUMENTS ADD COLUMN new_column TEXT")
            #     self.logger.info("버전 2로 마이그레이션 완료")
            # 추가 마이그레이션 로직 작성 가능
            
            # 최종적으로 user_version 업데이트
            # cursor.execute(f"PRAGMA user_version = {new_version}")
            # conn.commit()
            #self.logger.info(f"데이터베이스 마이그레이션 완료: {new_version}")            
        raise NotImplementedError("마이그레이션 로직이 구현되지 않았습니다.")

    def insert_text_content(self, 
                            file_name: str = "",
                            file_hash: str = "", 
                            total_pages: Optional[int] = 0, 
                            file_size: Optional[int] = 0, 
                            text_content: Optional[str] = "") -> bool:
        """
        마크다운 파일 정보를 데이터베이스에 저장

        Args:
            file_hash (str): 파일의 해시값
            file_name (str): 파일 이름 (필수, 고유)
            total_pages (Optional[int]): 총 페이지 수 (기본값: None)
            file_size (Optional[int]): 파일 크기 (기본값: None)
            text_content (Optional[str]): 파일의 전체 텍스트 콘텐츠 (기본값: "")

        Returns:
            bool: 저장 성공 여부
        """
        if not file_name:
            raise ValueError("file_name은 필수 항목이며 None 또는 빈 값일 수 없습니다.")
        if text_content is None:
            text_content = ""  # 최소한 빈 문자열이어야 함

        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                INSERT OR REPLACE INTO TB_DOCUMENTS
                (file_hash, file_name, total_pages, file_size, text_content, updated_at)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (file_hash, file_name, total_pages, file_size, text_content, datetime.now()))
            conn.commit()
            return True
        
    def get_document_by_hash(self, file_hash: str) -> Optional[Dict[str, Any]]:
        """
        파일 해시값으로 문서 정보 조회

        Args:
            file_hash (str): 조회할 파일의 해시값

        Returns:
            Optional[Dict[str, Any]]: 문서 정보 (없으면 None 반환)
        """
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT * FROM TB_DOCUMENTS WHERE file_hash = ?
            """, (file_hash,))
            row = cursor.fetchone()
            if row:
                return dict(row)
            return None

    def get_documents_all(self) -> List[Dict[str, Any]]:
        """
        데이터베이스의 모든 문서 조회
        
        Returns:
            List[Dict[str, Any]]: 문서 리스트 (file_name, file_hash, text_content 등)
        """
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT * FROM TB_DOCUMENTS
                ORDER BY updated_at DESC
            """)
            
            columns = [desc[0] for desc in cursor.description]
            results = [dict(zip(columns, row)) for row in cursor.fetchall()]
        
        return results

    def get_document_stats(self) -> Dict[str, Any]:
        """
        데이터베이스에 저장된 문서 통계 정보를 반환

        Returns:
            Dict[str, Any]: 문서 통계 정보
                - total_files: 총 파일 수
                - total_pages: 총 페이지 수
                - total_size_bytes: 총 파일 크기 (바이트 단위)
                - total_size_mb: 총 파일 크기 (MB 단위)
        """
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute('SELECT COUNT(*) FROM TB_DOCUMENTS')
            total_files = cursor.fetchone()[0]
            
            cursor.execute('SELECT SUM(total_pages) FROM TB_DOCUMENTS')
            total_pages = cursor.fetchone()[0] or 0
            
            cursor.execute('SELECT SUM(file_size) FROM TB_DOCUMENTS')
            total_size = cursor.fetchone()[0] or 0
            return {
                'total_files': total_files,
                'total_pages': total_pages,
                'total_size_bytes': total_size,
                'total_size_mb': round(total_size / (1024 * 1024), 2),
            }
            
    def search_documents(self, 
                        search_term: str, 
                        search_type: str = 'auto') -> List[Dict[str, Any]]:
        """
        파일명 또는 file_hash로 문서 검색
        
        Args:
            search_term (str): 검색어 (파일명 또는 file_hash)
            search_type (str): 검색 타입
                - 'auto': file_hash 형식(64자 hex)이면 hash 검색, 아니면 filename 검색
                - 'filename': file_name LIKE 검색
                - 'hash': file_hash 완전 일치 검색
        
        Returns:
            List[Dict[str, Any]]: 검색 결과 리스트
        
        Example:
            >>> # file_hash로 검색
            >>> results = db.search_documents("abc123def456...")
            >>> 
            >>> # 파일명으로 검색
            >>> results = db.search_documents("공고문", search_type='filename')
            >>> 
            >>> # 자동 판별
            >>> results = db.search_documents("test.pdf", search_type='auto')
        """
        if not search_term:
            self.logger.warning("검색어가 비어있습니다.")
            return []
        
        # search_type 결정
        if search_type == 'auto':
            # 64자 hex 문자열이면 hash 검색
            if len(search_term) == 64 and all(c in '0123456789abcdefABCDEF' for c in search_term):
                search_type = 'hash'
            else:
                search_type = 'filename'
        
        # 검색 쿼리 실행
        if search_type == 'hash':
            query = "SELECT * FROM TB_DOCUMENTS WHERE file_hash = ?"
            params = (search_term,)
        elif search_type == 'filename':
            query = "SELECT * FROM TB_DOCUMENTS WHERE file_name LIKE ?"
            params = (f"%{search_term}%",)
        else:
            self.logger.error(f"지원하지 않는 search_type: {search_type}")
            return []
        
        results = self.execute_query(query, params)
        self.logger.debug(f"검색 완료: {len(results)}건 ({search_type} 모드, 검색어: {search_term})")
        return results
            
    def execute_query(self, query: str, params: Optional[tuple] = None) -> List[Dict[str, Any]]:
        """
        SELECT 쿼리를 실행하고 결과를 반환
        
        Args:
            query (str): 실행할 SQL 쿼리
            params (Optional[tuple]): 쿼리 파라미터 (기본값: None)
        
        Returns:
            List[Dict[str, Any]]: 쿼리 결과 (각 행이 딕셔너리)
        
        Example:
            >>> results = db.execute_query("SELECT * FROM TB_DOCUMENTS WHERE file_hash = ?", ("abc123",))
            >>> for row in results:
            >>>     print(row['file_name'])
        """
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()
                if params:
                    cursor.execute(query, params)
                else:
                    cursor.execute(query)
                
                rows = cursor.fetchall()
                return [dict(row) for row in rows]
        
        except Exception as e:
            self.logger.error(f"쿼리 실행 중 오류: {e}")
            self.logger.debug(f"쿼리: {query}")
            if params:
                self.logger.debug(f"파라미터: {params}")
            return []
            
    def summary(self) -> None:
        """
        데이터베이스의 모든 테이블 요약 정보 출력.
        
        각 테이블의 컬럼명과 레코드 수를 로거로 출력합니다.
        """
        try:
            # 테이블 목록 조회
            tables = self.execute_query(
                "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
            )
            
            if not tables:
                self.logger.info("데이터베이스에 테이블이 없습니다.")
                return
            
            self.logger.info("=" * 80)
            self.logger.info(f"데이터베이스: {self.db_path}")
            self.logger.info(f"총 테이블 수: {len(tables)}")
            self.logger.info("=" * 80)
            
            for table_info in tables:
                table_name = table_info['name']
                
                # 컬럼 정보
                columns = self.execute_query(f"PRAGMA table_info({table_name})")
                column_names = [col['name'] for col in columns]
                
                # 레코드 수
                count_result = self.execute_query(f"SELECT COUNT(*) as cnt FROM {table_name}")
                row_count = count_result[0]['cnt'] if count_result else 0
                
                self.logger.info(f"테이블: {table_name}")
                self.logger.info(f"  - 컬럼 수: {len(column_names)}")
                self.logger.info(f"  - 컬럼명: {', '.join(column_names)}")
                self.logger.info(f"  - 레코드 수: {row_count}")
                self.logger.info("-" * 80)
                
        except Exception as e:
            self.logger.error(f"summary() 실행 중 오류: {e}")
            
logger.info("DocumentsDB 모듈이 로드되었습니다.")
if __name__ == "__main__":
    db = DocumentsDB("data/documents.db")
    db.summary()

# document_processer 테스트

In [1]:
# 환경 설정 및 모듈 임포트
import sys
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.processors.document_processor import DocumentProcessor
from src.db.documents_db import DocumentsDB
from src.utils.logging_config import get_logger

logger = get_logger(__name__)
logger.info("테스트 환경 설정 완료")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.
설정 검증 통과
2025-11-16 11:24:53 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-16 11:24:53 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-16 11:24:55 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-16 11:24:55 [I] __main__ - 테스트 환경 설정 완료


## 파일 하나만 넣기

In [6]:
# 하나만 넣기
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path
    
    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "pdf_conv_2"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: {len(pdf_files)}개")
    
    # 첫 번째 PDF 파일 처리
    pdf_path = pdf_files[0]
    logger.info(f"처리 중: {pdf_path.name}")
    
    try:
        processor = DocumentProcessor(db_path=str(project_root / "data" / "test_documents.db"))
        file_hash = processor.process_pdf(str(pdf_path))
        
        if file_hash:
            logger.info(f"✓ PDF 처리 성공: {file_hash[:16]}...")
        else:
            logger.error("✗ PDF 처리 실패 (PyMuPDF 미설치 가능성)")
            
    except Exception as e:
        logger.error(f"✗ PDF 처리 중 오류: {e}")

# 실행
test_real_pdf_if_available()

2025-11-14 14:00:26 [I] __main__ - 발견된 PDF 파일: 100개
2025-11-14 14:00:26 [I] __main__ - 처리 중: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:00:26 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: /home/spai0438/codeit_ai_g2b_search/data/test_documents.db)
2025-11-14 14:00:26 [I] [DOCP] - PDF 처리 시작: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:00:26 [D] [DOCP] - 파일 해시: 5a28516e24903a44...


PDF to Markdown:   0%|          | 0/96 [00:00<?, ?page/s]

2025-11-14 14:01:45 [I] [DOCP] - PDF 처리 완료: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf (96 페이지, 1675.0KB), 변환문자수=116,963
2025-11-14 14:01:45 [I] __main__ - ✓ PDF 처리 성공: 5a28516e24903a44...


## 전체 파일 로드해서 test_documents.db에 넣어보기

In [3]:
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path
    from tqdm import tqdm

    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "pdf_conv_3"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: **{len(pdf_files)}**개")
    
    # DocumentProcessor 인스턴스를 루프 밖에서 한 번만 생성
    try:
        processor = DocumentProcessor(db_path=str(project_root / "data" / "test_documents_2.db"))
    except Exception as e:
        logger.error(f"✗ DocumentProcessor 초기화 중 오류: {e}")
        return

    count = 1 # 문서 처리 개수 세는 카운트
    # --- 모든 PDF 파일을 처리하는 반복문 구조 ---
    for pdf_path in pdf_files:
        logger.info(f"{len(pdf_files)}개 파일 중 {count}번째 파일 처리 중:  **{pdf_path.name}**")
        
        try:
            file_hash = processor.process_pdf(str(pdf_path))
            
            if file_hash:
                logger.info(f"✓ PDF 처리 성공: {file_hash[:16]}...")
            else:
                # process_pdf가 성공했지만 해시를 반환하지 않은 경우 (예: 파일이 비어 있음)
                logger.error(f"✗ PDF 처리 실패 (파일 해시 반환 안 됨): {pdf_path.name}")
                
        except Exception as e:
            # PyMuPDF 미설치 등 구체적인 오류를 로깅
            logger.error(f"✗ PDF 처리 중 오류 [{pdf_path.name}]: {e}")
        count += 1
            
    logger.info("모든 PDF 파일 처리가 완료되었습니다.")

# 실행
test_real_pdf_if_available()

2025-11-16 13:41:27 [I] __main__ - 발견된 PDF 파일: **100**개
2025-11-16 13:41:27 [I] [DOCDB] - 데이터베이스 버전 초기화: 1
2025-11-16 13:41:27 [I] [DOCP] - 마커 덤프 활성화: data/markers
2025-11-16 13:41:27 [D] [DOCP] - DB에 문서 없음
2025-11-16 13:41:27 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: /home/spai0438/codeit_ai_g2b_search/data/test_documents_2.db)
2025-11-16 13:41:27 [I] __main__ - 100개 파일 중 1번째 파일 처리 중:  **한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf**
2025-11-16 13:41:27 [I] [DOCP] - PDF 처리 시작: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-16 13:41:27 [D] [DOCP] - 파일 해시: 4343a16648873581...
2025-11-16 13:41:27 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 4343a1664887358147d0fd51982ca273ac5d500e966b2cfcb990e1ac2437bb0f)


PDF to Markdown:   0%|          | 0/96 [00:00<?, ?page/s]

2025-11-16 13:42:33 [I] [DOCP] - PDF 처리 완료: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf (96 페이지, 1136.5KB), 변환문자수=148,690
2025-11-16 13:42:33 [I] __main__ - ✓ PDF 처리 성공: 4343a16648873581...
2025-11-16 13:42:33 [I] __main__ - 100개 파일 중 2번째 파일 처리 중:  **광주과학기술원_학사시스템 기능개선 사업.pdf**
2025-11-16 13:42:33 [I] [DOCP] - PDF 처리 시작: 광주과학기술원_학사시스템 기능개선 사업.pdf
2025-11-16 13:42:33 [D] [DOCP] - 파일 해시: ab7378a7405213f5...
2025-11-16 13:42:33 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: ab7378a7405213f53cf45a4f4ca4aabf22a38363c3331cadc9fddee266944e2f)


PDF to Markdown:   0%|          | 0/66 [00:00<?, ?page/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key i

...

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key i

PDF to Markdown:   0%|          | 0/137 [00:00<?, ?page/s]

2025-11-16 16:12:40 [I] [DOCP] - PDF 처리 완료: 재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf (137 페이지, 1265.9KB), 변환문자수=132,995
2025-11-16 16:12:40 [I] __main__ - ✓ PDF 처리 성공: 861bfd25152d39f2...
2025-11-16 16:12:40 [I] __main__ - 100개 파일 중 100번째 파일 처리 중:  **한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf**
2025-11-16 16:12:40 [I] [DOCP] - PDF 처리 시작: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf
2025-11-16 16:12:40 [D] [DOCP] - 파일 해시: a47caca5184d9eca...
2025-11-16 16:12:40 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: a47caca5184d9ecaf7bd6099cac61a2961f75ece3f7f9540a6bdabca2dbd5363)


PDF to Markdown:   0%|          | 0/74 [00:00<?, ?page/s]

2025-11-16 16:13:36 [I] [DOCP] - PDF 처리 완료: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf (74 페이지, 1626.8KB), 변환문자수=62,029
2025-11-16 16:13:36 [I] __main__ - ✓ PDF 처리 성공: a47caca5184d9eca...
2025-11-16 16:13:36 [I] __main__ - 모든 PDF 파일 처리가 완료되었습니다.


## 5. 실제 PDF 파일 테스트 (선택사항)

In [12]:
def verify_database_content() -> None:
    """DB 내용 확인 및 검증"""
    import sqlite3
    
    test_db_path = project_root / "data" / "test_documents.db"
    
    with sqlite3.connect(test_db_path) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        # 1. 파일 정보 조회
        # 모든 테이블 이름 조회 쿼리
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        logger.info(f"모든 테이블 이름 조회 쿼리 {cursor.fetchall()}")
        files = cursor.fetchall()
        logger.info(f"\n=== 파일 정보 ({len(files)}건) ===")
        for row in files:
            logger.info(f"  - {row['file_name']}: {row['total_pages']}페이지, {row['total_tokens']}토큰")
        
        # 2. 페이지 데이터 조회
        # cursor.execute("""
        #     SELECT file_hash, page_number, is_empty 
        #     FROM page_data 
        #     ORDER BY file_hash, page_number
        # """)
        pages = cursor.fetchall()
        logger.info(f"\n=== 페이지 데이터 ({len(pages)}건) ===")
        
        # 파일별로 그룹화
        from collections import defaultdict
        pages_by_file = defaultdict(list)
        for row in pages:
            pages_by_file[row['file_hash']].append(row)
        
        for file_hash, file_pages in pages_by_file.items():
            logger.info(f"  파일: {file_hash[:16]}...")
            for page in file_pages:
                status = "빈페이지" if page['is_empty'] else "정상"
                logger.info(f"    - 페이지 {page['page_number']}: {page['token_count']}토큰 ({status})")
        
        # 3. 통계 조회
        db = DocumentsDB(str(test_db_path))
        stats = db.get_document_stats()
        logger.info(f"\n=== 전체 통계 ===")
        logger.info(f"  총 파일: {stats['total_files']}개")
        logger.info(f"  총 페이지: {stats['total_pages']}개")
        # logger.info(f"  총 토큰: {stats['total_tokens']:,}개")
        logger.info(f"  총 크기: {stats['total_size_mb']}MB")
        
        logger.info("\n✓ DB 검증 완료")

# 실행
verify_database_content()

2025-11-14 14:20:49 [I] __main__ - 모든 테이블 이름 조회 쿼리 [<sqlite3.Row object at 0x7a2692c3f6a0>]
2025-11-14 14:20:49 [I] __main__ - 
=== 파일 정보 (0건) ===
2025-11-14 14:20:49 [I] __main__ - 
=== 페이지 데이터 (0건) ===
2025-11-14 14:20:49 [I] __main__ - 
=== 전체 통계 ===
2025-11-14 14:20:49 [I] __main__ -   총 파일: 1개
2025-11-14 14:20:49 [I] __main__ -   총 페이지: 96개
2025-11-14 14:20:49 [I] __main__ -   총 크기: 1.64MB
2025-11-14 14:20:49 [I] __main__ - 
✓ DB 검증 완료


## 4. DB 조회 및 검증

In [9]:
def test_pdf_processing_simulation() -> None:
    """PDF 처리 로직 시뮬레이션 (PyMuPDF 없이)"""
    
    # 실제 PDF 대신 더미 데이터로 로직 테스트
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # PDF 페이지 시뮬레이션
    dummy_pages = [
        "# 제목 페이지\n\n이것은 첫 번째 페이지입니다.",
        "# 본문 1\n\n두 번째 페이지 내용입니다.",
        "",  # 빈 페이지
        "# 결론\n\n마지막 페이지입니다."
    ]
    
    # 파일 정보 시뮬레이션
    dummy_hash = "dummy_pdf_hash_67890"
    dummy_filename = "test_dummy.pdf"
    
    logger.info(f"PDF 처리 시뮬레이션: {dummy_filename}")
    
    # 페이지별 처리
    all_content = []
    for page_num, page_text in enumerate(dummy_pages, start=1):
        is_empty = len(page_text.strip()) < 10
        
        if is_empty:
            page_content = "[빈 페이지]"
        else:
            page_content = f"--- 페이지 {page_num} ---\n\n{page_text}"
        
        all_content.append(page_content)
        
        # DB 저장
        token_count = processor.count_tokens(page_content)
        processor.docs_db.insert_page_data(
            file_hash=dummy_hash,
            page_number=page_num,
            markdown_content=page_content,
            token_count=token_count,
            is_empty=is_empty
        )
        logger.info(f"페이지 {page_num}: {token_count} 토큰, 빈페이지={is_empty}")
    
    # 전체 콘텐츠
    full_content = "\n\n".join(all_content)
    total_tokens = processor.count_tokens(full_content)
    
    # 파일 정보 저장
    processor.docs_db.insert_file_info(
        file_hash=dummy_hash,
        file_name=dummy_filename,
        total_pages=len(dummy_pages),
        file_size=len(full_content.encode('utf-8')),
        total_chars=len(full_content),
        total_tokens=total_tokens
    )
    
    logger.info(f"✓ PDF 시뮬레이션 완료: {len(dummy_pages)}페이지, {total_tokens}토큰")

# 테스트 실행
test_pdf_processing_simulation()

2025-11-08 22:36:33 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰


## 3. PDF 처리 테스트 (PyMuPDF 없이 시뮬레이션)

In [8]:
def test_document_processor_utils() -> None:
    """DocumentProcessor 유틸리티 함수 테스트"""
    import tempfile
    
    # DocumentProcessor 초기화
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # 1. 임시 파일 생성
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False, encoding='utf-8') as tf:
        test_content = "This is a test document for hash calculation."
        tf.write(test_content)
        temp_file_path = Path(tf.name)
    
    try:
        # 2. 파일 해시 계산
        file_hash = processor.calculate_file_hash(temp_file_path)
        logger.info(f"파일 해시: {file_hash[:16]}...")
        assert len(file_hash) == 64, "SHA-256 해시 길이 오류"
        
        # 3. 토큰 카운트
        token_count = processor.count_tokens(test_content)
        logger.info(f"토큰 수: {token_count}")
        assert token_count > 0, "토큰 카운트 오류"
        
        logger.info("✓ 유틸리티 함수 테스트 성공")
        
    finally:
        # 임시 파일 삭제
        if temp_file_path.exists():
            temp_file_path.unlink()

# 테스트 실행
test_document_processor_utils()

2025-11-08 22:36:26 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공


## 2. DocumentProcessor 해시 계산 및 토큰 카운트 테스트

In [7]:
def test_documents_db_crud() -> None:
    """DocumentsDB CRUD 테스트"""
    from datetime import datetime
    import sqlite3
    
    # 테스트용 DB 경로
    test_db_path = project_root / "data" / "test_documents.db"
    test_db_path.parent.mkdir(parents=True, exist_ok=True)
    
    # DB 초기화
    db = DocumentsDB(str(test_db_path))
    
    # 기존 테스트 데이터 삭제 (파일 삭제 대신)
    with sqlite3.connect(test_db_path) as conn:
        conn.execute("DELETE FROM page_data")
        conn.execute("DELETE FROM file_info")
        conn.commit()
    
    logger.info(f"테스트 DB 초기화: {test_db_path}")
    
    # 1. 파일 정보 삽입 (Create)
    test_hash = "test_hash_12345"
    success = db.insert_file_info(
        file_hash=test_hash,
        file_name="test_document.pdf",
        total_pages=3,
        file_size=1024 * 50,  # 50KB
        total_chars=5000,
        total_tokens=1200
    )
    logger.info(f"파일 정보 삽입: {success}")
    
    # 2. 페이지 데이터 삽입
    for page_num in range(1, 4):
        content = f"# 페이지 {page_num}\n\n테스트 콘텐츠입니다."
        db.insert_page_data(
            file_hash=test_hash,
            page_number=page_num,
            markdown_content=content,
            token_count=100 + page_num * 10,
            is_empty=False
        )
    logger.info("페이지 데이터 삽입 완료")
    
    # 3. 통계 조회 (Read)
    stats = db.get_document_stats()
    logger.info(f"문서 통계: {stats}")
    
    # 4. 데이터 확인
    assert stats['total_files'] >= 1, "파일 개수 불일치"
    assert stats['total_pages'] >= 3, "페이지 개수 불일치"
    
    logger.info("✓ CRUD 테스트 성공")
    return db, test_hash

# 테스트 실행
db, test_hash = test_documents_db_crud()

2025-11-08 22:36:20 [I] __main__ - 테스트 DB 초기화: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공


## 1. DocumentsDB 기본 CRUD 테스트

## 테스트 요약

위의 테스트들을 통해 다음을 확인했습니다:

1. **DocumentsDB CRUD**: 파일 정보/페이지 데이터 삽입 및 조회 정상 동작
2. **DocumentProcessor 유틸리티**: 파일 해시 계산, 토큰 카운트 정상
3. **PDF 처리 시뮬레이션**: 페이지별 콘텐츠 처리 로직 검증
4. **DB 검증**: 전체 통계 및 데이터 무결성 확인
5. **실제 PDF 처리**: PyMuPDF 사용 가능 시 자동 처리

**추가 테스트 방법**:
- `data/raw/` 디렉토리에 PDF 파일을 추가하면 실제 PDF 처리 테스트 자동 실행됩니다.
- HWP 파일 처리는 현재 `document_processor.py`에 구현되지 않았으므로 향후 추가 필요합니다.